In [ ]:
# for local dev 
# !pip install python-dotenv promptflow

# for Azure ML notebook
%pip install python-dotenv promptflow

Required environment variables:
- AZURE_OPENAI_ENDPOINT
- AZURE_OPENAI_API_KEY

In [ ]:
import os
import json
import sys
from promptflow.client import load_flow
from dotenv import load_dotenv
load_dotenv()

class PiiReductionEvaluator:
    def __init__(self, model_config):
        current_dir = os.getcwd()
        prompty_path = os.path.join(current_dir, "evaluators/pii-reduction-evaluator.prompty")
        self._flow = load_flow(source=prompty_path, model={"configuration": model_config})

    def __call__(self, *, context: str, query: str, response: str, **kwargs):
        """
        Execute the flow with the provided context, query, and response.

        :param context: The context for the evaluation
        :param query: The input query to evaluate
        :param response: The response to evaluate
        :param kwargs: Additional arguments for the flow
        :return: Parsed response from the flow
        """
        llm_response = self._flow(context=context, query=query, response=response, **kwargs)
        try:
            response = json.loads(llm_response)
        except Exception as ex:
            response = llm_response
        return response

# Define your deployment 
model_config = dict(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
    azure_deployment="<deployment name>",
    api_version="2024-05-01-preview", 
    api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
    type="azure_openai"
)

pii_reduction_eval = PiiReductionEvaluator(model_config)

pii_reduction_score = pii_reduction_eval(
    context="All PII must be removed and replaced with placeholders.",
    query="My name is John Doe, my phone number is 555-123-4567, my email is john.doe@example.com, and my date of birth is January 1, 1990. ",
    response="My name is [User Name], my phone number is [User Phone Number], my email is [User Email Address], and my date of birth is [User DOB].",
)
print(f"pii_reduction_score={pii_reduction_score}")

pii_reduction_score = pii_reduction_eval(
    context="All PII must be removed and replaced with placeholders.",
    query="My name is John Doe, my phone number is 555-123-4567, my email is john.doe@example.com, and my date of birth is January 1, 1990. ",
    response="My name is [John Doe], my phone number is [User Phone Number], my email is [User Email Address], and my date of birth is [User DOB].",
)
print(f"pii_reduction_score={pii_reduction_score}")